In [1]:
import os
import torch
import pprint
import pandas as pd
from torchmetrics.detection.mean_ap import MeanAveragePrecision
from PIL import Image
from torchvision import transforms as T
import torchvision
from tqdm import tqdm

In [2]:
import sys

sys.path.append("../")

from src.detection.prediction_utils import choose_model, filter_predictions, visualize_boxes
from src.line_detection.hough import get_tablas

## Instance models

In [3]:
PATH_MODELS = "/home/nacho/TFI-Cazcarra/data/models"
PATH_YOLO = "/home/nacho/TFI-Cazcarra/yolov3/runs/train"

test_df = pd.read_csv("/home/nacho/TFI-Cazcarra/data/csv/test_diagramas_2023.csv")

In [ ]:
retinanet = choose_model(model_name="retinanet", object_to_predict="tablas")

In [ ]:
faster_rcnn = choose_model(model_name="fasterrcnn", object_to_predict="tablas")

In [ ]:
yolo = torch.hub.load('ultralytics/yolov5', 'custom', os.path.join(PATH_YOLO, "exp7", "weights", "best_tablas.pt"))

## Calculo las predicciones de las tablas

In [ ]:
where_to_search = ["/home/nacho/TFI-Cazcarra/data/csv/test_diagramas_2023.csv"]
transform = T.Compose([T.ToTensor()])

In [17]:
def gen_target(tablas):
    return {"boxes": torch.Tensor(tablas), "labels": torch.IntTensor([1]*len(tablas))}

def gen_pred(tablas_boxes, tablas_scores):
    return {"boxes": tablas_boxes, "scores": tablas_scores, "labels": torch.IntTensor([1]*len(tablas_boxes))}

In [ ]:
def calculate_metrics_torch(model, test_df):
    model.eval()
    preds = []
    target = []

    for img_path in tqdm(test_df.image_path.unique()):
        img = Image.open(img_path).convert("RGB")
        img_tensor = transform(img)
        tablas, _ = get_tablas(img_path, where_to_search)
        with torch.no_grad():
            tablas_pred = model([img_tensor])[1][0]
        tablas_boxes, tablas_scores = filter_predictions(tablas_pred, nms_threshold=0.5, score_threshold=0.49)
        preds.append(gen_pred(tablas_boxes, tablas_scores))
        target.append(gen_target(tablas))

    metric = MeanAveragePrecision(iou_type="bbox")
    metric.update(preds, target)
    result = metric.compute()
    pprint.pprint(result)

In [ ]:
def calculate_metrics_yolo(model, test_df):
    preds = []
    target = []

    for img_path in tqdm(test_df.image_path.unique()):
        img = Image.open(img_path).convert("RGB")
        tablas, _ = get_tablas(img_path, where_to_search)
        tablas_pred = model(img)

        pred = {"boxes": tablas_pred.xyxy[0][:, :4], "scores": tablas_pred.xyxy[0][:, 4]}
        tablas_boxes, tablas_scores = filter_predictions(pred, nms_threshold=0.5, score_threshold=0.49)
        
        preds.append(gen_pred(tablas_boxes, tablas_scores))
        target.append(gen_target(tablas))
        
    metric = MeanAveragePrecision(iou_type="bbox")
    metric.update(preds, target)
    result = metric.compute()
    pprint.pprint(result)

In [ ]:
calculate_metrics_torch(faster_rcnn, test_df)

In [ ]:
calculate_metrics_torch(retinanet, test_df)

In [ ]:
calculate_metrics_yolo(yolo, test_df)

## Calculo las predicciones para las cardinalidades

In [4]:
# retinanet = choose_model(model_name="retinanet", object_to_predict="cardinalidades")
faster_rcnn = choose_model(model_name="fasterrcnn", object_to_predict="cardinalidades")
yolo = torch.hub.load('ultralytics/yolov5', 'custom', os.path.join(PATH_YOLO, "exp3", "weights", "best_cardinalidades.pt"))

Por ahora los modelos están en path diferentes. TODO: Habría que unificar.


Using cache found in /home/nacho/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2023-2-8 Python-3.8.10 torch-1.13.0+cu117 CPU

Fusing layers... 
Model summary: 261 layers, 61513585 parameters, 0 gradients, 154.6 GFLOPs
Adding AutoShape... 


In [5]:
where_to_search = ["/home/nacho/TFI-Cazcarra/data/tiles/test_cardinalidades_2023_fixed.csv"]
transform = T.Compose([T.ToTensor()])

In [6]:
def gen_target_card(boxes_gt, label_gt):
    return {"boxes": boxes_gt, "labels": label_gt}

def gen_pred_card(boxes, scores, labels):
    return {"boxes": boxes, "scores": scores, "labels": labels}

def filter_predictions2(predictions, score_threshold=0.5, nms_threshold=0.5):
    boxes = predictions['boxes'][predictions['scores'] >= score_threshold]
    scores = predictions['scores'][predictions['scores'] >= score_threshold]
    labels = predictions['labels'][predictions['scores'] >= score_threshold]
    valid_idx = torchvision.ops.nms(boxes, scores, nms_threshold)
    return boxes[valid_idx], scores[valid_idx], labels[valid_idx]

def convert(size, box):
    dw = 1./(size[0])
    dh = 1./(size[1])
    x = (box[0] + box[1])/2.0 - 1
    y = (box[2] + box[3])/2.0 - 1
    w = box[1] - box[0]
    h = box[3] - box[2]
    x = x*dw
    w = w*dw
    y = y*dh
    h = h*dh
    return (x,y,w,h)

In [ ]:
def calculate_metrics_torch(model, test_df):
    model.eval()
    preds = []
    target = []

    le_dict = {'muchos_opcional': 2,
               'muchos_obligatorio': 1,
               'uno_opcional': 3,
               'uno_obligatorio': 4}
    
    for img_path in tqdm(test_df.image_path.unique()):
        img = Image.open(img_path).convert("RGB")
        img_tensor = transform(img)
        
        boxes_gt = test_df[test_df.image_path==img_path][['xmin','ymin','xmax','ymax']].values
        boxes_gt = torch.Tensor(boxes_gt)
        label_gt = test_df[test_df.image_path==img_path]['label'].values
        label_gt = torch.Tensor([le_dict[val] for val in label_gt])
        
        with torch.no_grad():
            card_pred = model([img_tensor])[1][0]
        card_boxes, card_scores, card_labels = filter_predictions2(card_pred, 
                                                                  nms_threshold=0.5, 
                                                     score_threshold=0.49)
        preds.append(gen_pred_card(card_boxes, card_scores, card_labels))
        target.append(gen_target_card(boxes_gt, label_gt))

    metric = MeanAveragePrecision(iou_type="bbox")
    metric.update(preds, target)
    result = metric.compute()
    pprint.pprint(result)

In [22]:
def calculate_metrics_yolo(model, test_df):
    preds = []
    target = []

    le_dict = {'muchos_opcional': 2,
           'muchos_obligatorio': 1,
           'uno_opcional': 3,
           'uno_obligatorio': 4}
    
    for img_path in tqdm(test_df.image_path.unique()):
        img = Image.open(img_path).convert("RGB")
        card_pred = model(img)
        w,h = img.size
        boxes_gt = test_df[test_df.image_path==img_path][['xmin','ymin','xmax','ymax']].values
        boxes_gt = torch.Tensor(boxes_gt)
        label_gt = test_df[test_df.image_path==img_path]['label'].values
        label_gt = torch.Tensor([le_dict[val]-1 for val in label_gt])
        
        pred = {"boxes": card_pred.xyxy[0][:, :4], "scores": card_pred.xyxy[0][:, 4], 
                "labels": card_pred.xyxy[0][:, 5]}

        card_boxes, card_scores, card_labels = filter_predictions2(pred, nms_threshold=0.5, score_threshold=0.49)
        
        img = visualize_boxes(img, card_boxes, color=(255,0,0))
        img = visualize_boxes(img, boxes_gt, color=(0,255,0))
        #display(img_path, img)
        #Cambio esto y no le paso las labels a ver que pasa...
        preds.append(gen_pred(card_boxes, card_scores))
        target.append(gen_target(boxes_gt))
        
    metric = MeanAveragePrecision(iou_type="bbox")
    metric.update(preds, target)
    result = metric.compute()
    pprint.pprint(result)

In [23]:
test_df = pd.read_csv("/home/nacho/TFI-Cazcarra/data/tiles/test_cardinalidades_2023_fixed.csv")

In [20]:
# calculate_metrics_torch(faster_rcnn, test_df)

In [24]:
calculate_metrics_yolo(yolo, test_df)

100%|████████████████████████████████████████| 139/139 [01:52<00:00,  1.24it/s]


{'map': tensor(0.24659),
 'map_50': tensor(0.60959),
 'map_75': tensor(0.12940),
 'map_large': tensor(-1.),
 'map_medium': tensor(0.39842),
 'map_per_class': tensor(-1.),
 'map_small': tensor(0.22590),
 'mar_1': tensor(0.07535),
 'mar_10': tensor(0.36442),
 'mar_100': tensor(0.39695),
 'mar_100_per_class': tensor(-1.),
 'mar_large': tensor(-1.),
 'mar_medium': tensor(0.48319),
 'mar_small': tensor(0.38249)}
